<a href="https://colab.research.google.com/github/ManjuRama/FinMath/blob/main/TFR_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
n_hours = 24  # Number of past hours to use for predicting the next hour

# Define the sliding window function for time series data
def create_sequences(df, n_hours):
    X = []
    y = []
    for i in range(len(df) - n_hours):
        X.append(df.iloc[i:i+n_hours].values)  # n_hours past records as input
        y.append(df.iloc[i+n_hours]['Adj. Close'])  # Next hour's 'Adj. Close' as target
    return np.array(X), np.array(y)

# Prepare input features and target variables for training
X_train_data, y_train_data = create_sequences(df_train, n_hours)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train_data).float()
y_train = torch.tensor(y_train_data).float().unsqueeze(1)  # Shape (batch_size, 1)

# Update DataLoader to handle sequence data (batch_size, seq_len, num_features)
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, num_tickers, embedding_dim, num_heads, ff_dim):
        super(TransformerModel, self).__init__()

        # Embedding for ticker
        self.ticker_embedding = nn.Embedding(num_tickers, embedding_dim)

        # Multi-Head Attention layer for sequence input
        self.multihead_attn = nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=num_heads, batch_first=True)

        # Feed Forward Network
        self.ffn = nn.Sequential(
            nn.Linear(embedding_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embedding_dim)
        )

        # Layer normalization
        self.layer_norm1 = nn.LayerNorm(embedding_dim)
        self.layer_norm2 = nn.LayerNorm(embedding_dim)

        # Final output layer for regression ('Adj. Close')
        self.output_layer = nn.Linear(embedding_dim, 1)

    def forward(self, inputs):
        # Inputs is now a sequence of shape (batch_size, seq_len, num_features)
        ticker_input = inputs[:, :, 0].long()  # Assuming ticker is the first feature in sequence
        non_ticker_inputs = inputs[:, :, 1:]  # The rest of the features

        # Embedding for ticker
        ticker_embedded = self.ticker_embedding(ticker_input)  # Shape: (batch_size, seq_len, embedding_dim)

        # Combine ticker embedding with other inputs
        combined_input = torch.cat([ticker_embedded, non_ticker_inputs], dim=2)  # Shape: (batch_size, seq_len, embedding_dim + other features)

        # Multi-head attention over the sequence
        attn_output, _ = self.multihead_attn(combined_input, combined_input, combined_input)

        # Add & normalize
        attn_output = self.layer_norm1(attn_output + combined_input)

        # Feed-forward network
        ffn_output = self.ffn(attn_output)

        # Add & normalize
        transformer_output = self.layer_norm2(ffn_output + attn_output)

        # Take the output corresponding to the last hour in the sequence for final prediction
        output = transformer_output[:, -1, :]  # Shape: (batch_size, embedding_dim)
        output = self.output_layer(output)  # Shape: (batch_size, 1)

        return output


In [ ]:
# Training loop
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, targets in train_loader:
            # Forward pass
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')


In [ ]:
# Prepare test data similarly using the sliding window approach
X_test_data, y_test_data = create_sequences(df_test, n_hours)

# Convert to PyTorch tensors
X_test = torch.tensor(X_test_data).float()
y_test = torch.tensor(y_test_data).float().unsqueeze(1)

# Create test DataLoader
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Forecasting the next hour
model.eval()
with torch.no_grad():
    y_pred_list = []
    for inputs, _ in test_loader:
        outputs = model(inputs)
        y_pred_list.append(outputs)

    y_pred = torch.cat(y_pred_list).numpy()

# Calculate MSE for the test set
mse = mean_squared_error(y_test.numpy(), y_pred)
print("Test Loss (MSE):", mse)
